<a href="https://colab.research.google.com/github/PedroColima2026/CursoTec/blob/main/Ev_diagnosticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cuestionarios


In [ ]:
# @title 1.1 Reporte grupal
!pip install fpdf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF
from google.colab import files

# Función 1: Cargar archivo
def cargar_archivo():
    print("Carga el archivo desde tu escritorio:")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    return pd.read_excel(file_name)

# Función 2: Identificar preguntas
def identificar_preguntas(df):
    preguntas_cols = [col for col in df.columns if 'Pregunta' in col]
    print(f"\nSe identificaron {len(preguntas_cols)} columnas con la palabra 'Pregunta':")
    print(preguntas_cols)
    return preguntas_cols

# Función 3: Solicitar información del docente
def solicitar_informacion_docente():
    print("Por favor, ingresa la información del docente:")
    nombre = input("Nombre del docente: ")
    asignatura = input("Asignatura: ")
    aula = input("Aula: ")
    carrera = input("Carrera: ")
    return {"Nombre": nombre, "Asignatura": asignatura, "Aula": aula, "Carrera": carrera}

# Función 4: Solicitar respuestas correctas
def solicitar_respuestas(preguntas_cols):
    respuestas_correctas = {}
    for pregunta in preguntas_cols:
        respuesta = input(f"Ingrese la respuesta correcta para '{pregunta}': ")
        respuestas_correctas[pregunta] = respuesta
    return respuestas_correctas

# Función 5: Calcular aciertos generales
def calcular_aciertos_generales(df, preguntas_cols, respuestas_correctas):
    df['Aciertos'] = df[preguntas_cols].apply(
        lambda row: sum(1 if row[col] == respuestas_correctas[col] else 0 for col in preguntas_cols),
        axis=1
    )
    return df

# Función 6: Generar diagrama de cajas y bigotes
def generar_diagrama(df):
    plt.figure(figsize=(10, 6))
    plt.boxplot(df['Aciertos'], vert=False, patch_artist=True, labels=['Aciertos'])
    plt.title("Diagrama de Caja y Bigotes: Aciertos Generales")
    plt.xlabel("Número de Aciertos")
    plt.grid(axis='y')
    plt.savefig("boxplot_aciertos_generales.png")
    plt.close()

# Función 7: Clasificar participantes según bigotes, interior de la caja y datos atípicos
def clasificar_participantes(df):
    Q1 = np.percentile(df['Aciertos'], 25)
    Q3 = np.percentile(df['Aciertos'], 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    clasificaciones = {
        'Interior de la Caja (Rango Intercuartil)': df[(df['Aciertos'] >= Q1) & (df['Aciertos'] <= Q3)],
        'En los Bigotes': df[(df['Aciertos'] >= lower_bound) & (df['Aciertos'] < Q1) | (df['Aciertos'] > Q3) & (df['Aciertos'] <= upper_bound)],
        'Datos Atípicos': df[(df['Aciertos'] < lower_bound) | (df['Aciertos'] > upper_bound)]
    }

    for clasificacion, datos in clasificaciones.items():
        print(f"\nParticipantes en {clasificacion}:")
        if not datos.empty:
            print(datos[['Dirección de correo electrónico', 'Aciertos']])
        else:
            print(f"No se encontraron participantes en {clasificacion}.")

    return clasificaciones

# Función 8: Generar reporte en PDF
def generar_reporte_pdf(docente_info, clasificaciones):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", style="B", size=14)
    pdf.cell(200, 10, txt="Reporte de Evaluación General", ln=True, align="C")
    pdf.ln(10)

    # Información del docente
    pdf.set_font("Arial", size=12)
    for key, value in docente_info.items():
        pdf.cell(0, 10, f"{key}: {value}", ln=True)
    pdf.ln(10)

    # Diagrama de caja y bigotes
    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Diagrama de Caja y Bigotes", ln=True)
    pdf.image("boxplot_aciertos_generales.png", x=10, y=None, w=190)
    pdf.ln(10)

    # Participantes por clasificaciones
    for clasificacion, datos in clasificaciones.items():
        pdf.set_font("Arial", style="B", size=12)
        pdf.cell(0, 10, f"Participantes en {clasificacion}", ln=True)
        pdf.set_font("Arial", size=10)
        if not datos.empty:
            for index, row in datos.iterrows():
                pdf.cell(0, 10, f"{row['Dirección de correo electrónico']}: {row['Aciertos']}", ln=True)
        else:
            pdf.cell(0, 10, "No se encontraron participantes en esta clasificación.", ln=True)
        pdf.ln(10)

    # Guardar el PDF
    pdf_file_name = "Reporte_Evaluacion_General.pdf"
    pdf.output(pdf_file_name)
    print(f"\nReporte generado: {pdf_file_name}")
    files.download(pdf_file_name)

# Flujo principal
def flujo_principal():
    df = cargar_archivo()
    preguntas_cols = identificar_preguntas(df)
    docente_info = solicitar_informacion_docente()
    respuestas_correctas = solicitar_respuestas(preguntas_cols)
    df = calcular_aciertos_generales(df, preguntas_cols, respuestas_correctas)
    generar_diagrama(df)
    clasificaciones = clasificar_participantes(df)
    generar_reporte_pdf(docente_info, clasificaciones)

# Ejecutar el flujo principal
flujo_principal()

In [ ]:
# @title 1.2 Reporte individual por categorias
!pip install fpdf
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fpdf import FPDF

# Función 1: Solicitar información del docente
def solicitar_informacion_docente():
    print("Por favor, ingresa la información del docente:")
    nombre = input("Nombre del docente: ")
    asignatura = input("Asignatura: ")
    aula = input("Aula: ")
    carrera = input("Carrera: ")
    return {"Nombre": nombre, "Asignatura": asignatura, "Aula": aula, "Carrera": carrera}

# Función 2: Cargar archivo
def cargar_archivo():
    print("Carga el archivo desde tu escritorio:")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    return pd.read_excel(file_name)

# Función 3: Identificar preguntas
def identificar_preguntas(df):
    preguntas_cols = [col for col in df.columns if 'Pregunta' in col]
    print(f"\nSe identificaron {len(preguntas_cols)} columnas con la palabra 'Pregunta':")
    print(preguntas_cols)
    return preguntas_cols

# Función 4: Definir categorías
def definir_categorias():
    categorias = {}
    num_categorias = int(input("\n¿Cuántas categorías quieres definir?: "))
    for i in range(num_categorias):
        nombre_categoria = input(f"Ingresa el nombre de la categoría {i+1}: ")
        preguntas_categoria = input(f"Ingrese los números de las preguntas para '{nombre_categoria}' separadas por comas (e.g., 1, 2, 3): ")
        categorias[nombre_categoria] = [f"Pregunta {int(num.strip())}" for num in preguntas_categoria.split(",")]
    print("\nCategorías definidas:")
    for categoria, preguntas in categorias.items():
        print(f"{categoria}: {preguntas}")
    return categorias

# Función 5: Solicitar respuestas correctas
def solicitar_respuestas(preguntas_cols):
    respuestas_correctas = {}
    for pregunta in preguntas_cols:
        respuesta = input(f"Ingrese la respuesta correcta para '{pregunta}': ")
        respuestas_correctas[pregunta] = respuesta
    return respuestas_correctas

# Función 6: Calcular aciertos por categoría
def calcular_aciertos(df, preguntas_cols, categorias, respuestas_correctas):
    for categoria, preguntas in categorias.items():
        df[categoria] = df[preguntas].apply(
            lambda row: sum(1 if row[col] == respuestas_correctas[col] else 0 for col in preguntas),
            axis=1
        )
    df['Aciertos_totales'] = df[list(categorias.keys())].sum(axis=1)
    return df

# Función 7: Generar reporte en PDF
def generar_reporte(docente_info, df, categorias):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    # Título del reporte
    pdf.set_font("Arial", style="B", size=14)
    pdf.cell(200, 10, txt="Reporte de Resultados de Evaluación", ln=True, align="C")
    pdf.ln(10)

    # Información del docente
    pdf.set_font("Arial", size=12)
    for key, value in docente_info.items():
        pdf.cell(0, 10, f"{key}: {value}", ln=True)
    pdf.ln(10)

    # Diagrama de caja y bigotes
    plt.figure(figsize=(10, 6))
    data_boxplot = [df[categoria] for categoria in categorias.keys()]
    plt.boxplot(data_boxplot, patch_artist=True, labels=categorias.keys())
    plt.title("Diagrama de Caja y Bigotes por Categoría")
    plt.ylabel("Puntaje")
    plt.xlabel("Categorías")
    plt.grid(axis='y')
    plt.savefig("boxplot.png")
    plt.close()

    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Diagrama de Caja y Bigotes por Categoría", ln=True)
    pdf.image("boxplot.png", x=10, y=None, w=190)
    pdf.ln(10)

    # Datos por categoría
    pdf.set_font("Arial", size=12)
    for categoria in categorias.keys():
        Q1 = np.percentile(df[categoria], 25)
        Q3 = np.percentile(df[categoria], 75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        datos_bigotes = df[(df[categoria] >= lower_bound) & (df[categoria] <= upper_bound)]
        datos_atipicos = df[(df[categoria] < lower_bound) | (df[categoria] > upper_bound)]

        pdf.set_font("Arial", style="B", size=12)
        pdf.cell(0, 10, f"Participantes en los Bigotes - {categoria}", ln=True)
        if not datos_bigotes.empty:
            pdf.set_font("Arial", size=10)
            for index, row in datos_bigotes.iterrows():
                pdf.cell(0, 10, f"{row['Dirección de correo electrónico']}: {row[categoria]}", ln=True)
        else:
            pdf.cell(0, 10, "No se encontraron participantes en los bigotes.", ln=True)
        pdf.ln(10)

        pdf.set_font("Arial", style="B", size=12)
        pdf.cell(0, 10, f"Valores Atípicos - {categoria}", ln=True)
        if not datos_atipicos.empty:
            pdf.set_font("Arial", size=10)
            for index, row in datos_atipicos.iterrows():
                pdf.cell(0, 10, f"{row['Dirección de correo electrónico']}: {row[categoria]}", ln=True)
        else:
            pdf.cell(0, 10, "No se encontraron valores atípicos.", ln=True)
        pdf.ln(10)

    # Guardar el PDF
    pdf_file_name = "Reporte_Resultados.pdf"
    pdf.output(pdf_file_name)
    print(f"\nReporte generado: {pdf_file_name}")
    files.download(pdf_file_name)

# Flujo principal
def flujo_principal():
    docente_info = solicitar_informacion_docente()
    df = cargar_archivo()
    preguntas_cols = identificar_preguntas(df)
    categorias = definir_categorias()
    respuestas_correctas = solicitar_respuestas(preguntas_cols)
    df = calcular_aciertos(df, preguntas_cols, categorias, respuestas_correctas)
    generar_reporte(docente_info, df, categorias)

# Ejecutar el flujo principal
flujo_principal()


# 2. Escalas

In [ ]:
# @title 2.1 Suma de puntajes
from google.colab import files
import pandas as pd
from fpdf import FPDF

# Paso 1: Cargar archivo de Excel desde escritorio
def cargar_archivo():
    print("Carga el archivo desde tu escritorio:")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    return pd.read_excel(file_name)

# Paso 2: Solicitar datos del docente y categorías
def solicitar_datos_docente():
    print("Por favor, ingresa la información del docente:")
    nombre = input("Nombre del docente: ")
    asignatura = input("Nombre de la asignatura: ")
    aula = input("Aula: ")
    carrera = input("Carrera: ")
    escala = input("Nombre de la escala analizada: ")
    return {"Nombre": nombre, "Asignatura": asignatura, "Aula": aula, "Carrera": carrera, "Escala": escala}

def solicitar_categorias():
    categorias = {}
    num_categorias = int(input("\n¿Cuántas categorías tiene la escala?: "))
    for i in range(num_categorias):
        nombre_categoria = input(f"Ingresa el nombre de la categoría {i+1}: ")
        preguntas_categoria = input(f"Ingrese los números de las preguntas para '{nombre_categoria}' separadas por comas (e.g., 1, 2, 3): ")
        categorias[nombre_categoria] = [int(num.strip()) for num in preguntas_categoria.split(",")]
    print("\nCategorías definidas:")
    for categoria, preguntas in categorias.items():
        print(f"{categoria}: {preguntas}")
    return categorias

# Paso 3: Calcular magnitudes
def calcular_magnitudes(data, categorias):
    resultados = pd.DataFrame()
    resultados['Folio'] = data['Folio']
    for categoria, preguntas in categorias.items():
        columnas_preguntas = [f'Pregunta {num}' for num in preguntas]
        resultados[categoria] = data[columnas_preguntas].sum(axis=1)
    return resultados

# Paso 4: Generar semáforo de resiliencia
def generar_semaforo(resultados, categorias):
    niveles = ['Nula', 'Baja', 'Media', 'Alta', 'Total']
    for categoria in categorias.keys():
        max_puntaje = resultados[categoria].max()
        umbrales = [max_puntaje * i / 5 for i in range(1, 6)]

        def asignar_semaforo(valor):
            if valor <= umbrales[0]:
                return niveles[0]
            elif valor <= umbrales[1]:
                return niveles[1]
            elif valor <= umbrales[2]:
                return niveles[2]
            elif valor <= umbrales[3]:
                return niveles[3]
            else:
                return niveles[4]

        resultados[f'Semáforo {categoria}'] = resultados[categoria].apply(asignar_semaforo)
    return resultados

# Paso 5: Generar tablas solicitadas
def generar_tablas(resultados, categorias):
    nula_baja_por_categoria = {}
    for categoria in categorias.keys():
        nula_baja = resultados[resultados[f'Semáforo {categoria}'].isin(['Nula', 'Baja'])]
        nula_baja_por_categoria[categoria] = nula_baja[['Folio', f'Semáforo {categoria}']]

    resultados['Nula_Baja_Total'] = resultados[[f'Semáforo {categoria}' for categoria in categorias.keys()]].apply(
        lambda row: sum(1 for nivel in row if nivel in ['Nula', 'Baja']), axis=1
    )
    nula_baja_multiple = resultados[resultados['Nula_Baja_Total'] > 1][['Folio', 'Nula_Baja_Total']]

    return nula_baja_por_categoria, nula_baja_multiple

# Paso 6: Generar reporte PDF
def generar_reporte_pdf(docente_info, nula_baja_por_categoria, nula_baja_multiple):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", style="B", size=14)
    pdf.cell(200, 10, txt="Reporte de capacidad", ln=True, align="C")
    pdf.ln(10)

    # Información del docente
    pdf.set_font("Arial", size=12)
    for key, value in docente_info.items():
        pdf.cell(0, 10, f"{key}: {value}", ln=True)
    pdf.ln(10)

    # Tablas de Nula o Baja Resiliencia por Categoría
    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Nula o Baja capacidad por Categoría", ln=True)
    pdf.set_font("Arial", size=10)
    for categoria, tabla in nula_baja_por_categoria.items():
        pdf.cell(0, 10, f"Categoría: {categoria}", ln=True)
        if not tabla.empty:
            for _, row in tabla.iterrows():
                pdf.cell(0, 10, f"Folio: {row['Folio']} - Nivel: {row[f'Semáforo {categoria}']}", ln=True)
        else:
            pdf.cell(0, 10, "No se encontraron participantes en este nivel.", ln=True)
        pdf.ln(5)

    # Tabla de Nula o Baja Resiliencia en Más de una Categoría
    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Nula o Baja capacidad en Más de una Categoría", ln=True)
    pdf.set_font("Arial", size=10)
    if not nula_baja_multiple.empty:
        for _, row in nula_baja_multiple.iterrows():
            pdf.cell(0, 10, f"Folio: {row['Folio']} - Total Categorías: {row['Nula_Baja_Total']}", ln=True)
    else:
        pdf.cell(0, 10, "No se encontraron participantes en más de una categoría.", ln=True)

    # Guardar el PDF
    pdf_file_name = "Reporte_Capacidad.pdf"
    pdf.output(pdf_file_name)
    print(f"\nReporte generado: {pdf_file_name}")
    files.download(pdf_file_name)

# Flujo principal
def flujo_principal():
    data = cargar_archivo()
    docente_info = solicitar_datos_docente()
    categorias = solicitar_categorias()
    resultados = calcular_magnitudes(data, categorias)
    resultados = generar_semaforo(resultados, categorias)

    # Generar tablas
    nula_baja_por_categoria, nula_baja_multiple = generar_tablas(resultados, categorias)

    # Generar PDF
    generar_reporte_pdf(docente_info, nula_baja_por_categoria, nula_baja_multiple)

# Ejecutar el flujo principal
flujo_principal()

In [ ]:
# @title 2.2 Rango de valores
from google.colab import files
import pandas as pd
from fpdf import FPDF

# Paso 1: Cargar archivo de Excel desde escritorio
def cargar_archivo():
    print("Carga el archivo desde tu escritorio:")
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
    return pd.read_excel(file_name)

# Paso 2: Solicitar información del usuario y diagnósticos
def solicitar_informacion_usuario():
    print("Por favor, ingresa la información del docente:")
    nombre = input("Nombre del docente: ")
    asignatura = input("Nombre de la asignatura: ")
    aula = input("Aula: ")
    carrera = input("Carrera: ")
    return {"Nombre": nombre, "Asignatura": asignatura, "Aula": aula, "Carrera": carrera}

def solicitar_diagnosticos():
    diagnosticos = {}
    num_diagnosticos = int(input("\n¿Cuántos diagnósticos primarios hay?: "))
    for i in range(num_diagnosticos):
        nombre = input(f"Ingresa el nombre del diagnóstico {i+1}: ")
        rango_min = float(input(f"Ingrese el puntaje mínimo para '{nombre}': "))
        rango_max = float(input(f"Ingrese el puntaje máximo para '{nombre}': "))
        diagnosticos[nombre] = (rango_min, rango_max)
    print("\nDiagnósticos primarios definidos:")
    for diag, rango in diagnosticos.items():
        print(f"{diag}: {rango[0]} - {rango[1]} puntos")
    return diagnosticos

# Paso 3: Identificar columnas con el nombre "Pregunta"
def identificar_columnas_pregunta(df):
    preguntas_cols = [col for col in df.columns if 'Pregunta' in col]
    print(f"\nSe identificaron {len(preguntas_cols)} columnas con la palabra 'Pregunta':")
    print(preguntas_cols)
    return preguntas_cols

# Paso 4: Calcular suma de las columnas con "Pregunta" por fila
def calcular_suma(df, preguntas_cols):
    df['Suma Total'] = df[preguntas_cols].iloc[1:].sum(axis=1)
    return df

# Paso 5: Asignar diagnóstico primario
def asignar_diagnostico(df, diagnosticos):
    def asignar(row):
        for diag, (min_val, max_val) in diagnosticos.items():
            if min_val <= row['Suma Total'] <= max_val:
                return diag
        return "Sin Diagnóstico"

    df['Diagnóstico Primario'] = df.apply(asignar, axis=1)
    return df

# Paso 6: Generar reporte en PDF
def generar_reporte_pdf(usuario_info, df, diagnosticos):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", style="B", size=14)
    pdf.cell(200, 10, txt="Reporte de Diagnósticos Primarios", ln=True, align="C")
    pdf.ln(10)

    # Información del usuario
    pdf.set_font("Arial", size=12)
    for key, value in usuario_info.items():
        pdf.cell(0, 10, f"{key}: {value}", ln=True)
    pdf.ln(10)

    # Diagnósticos y rangos
    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Rangos de Diagnósticos", ln=True)
    pdf.set_font("Arial", size=10)
    for diag, (min_val, max_val) in diagnosticos.items():
        pdf.cell(0, 10, f"{diag}: {min_val} - {max_val} puntos", ln=True)
    pdf.ln(10)

    # Tabla de Folios por Diagnóstico
    pdf.set_font("Arial", style="B", size=12)
    pdf.cell(0, 10, "Folios Agrupados por Diagnóstico", ln=True)
    pdf.set_font("Arial", size=10)
    diagnosticos_unicos = df['Diagnóstico Primario'].unique()
    for diag in diagnosticos_unicos:
        pdf.cell(0, 10, f"Diagnóstico: {diag}", ln=True)
        folios = df[df['Diagnóstico Primario'] == diag]['Folio'].tolist()
        for folio in folios:
            pdf.cell(0, 10, f"- Folio: {folio}", ln=True)
        pdf.ln(5)

    # Guardar el PDF
    pdf_file_name = "Reporte_Diagnosticos_Primarios.pdf"
    pdf.output(pdf_file_name)
    print(f"\nReporte generado: {pdf_file_name}")
    files.download(pdf_file_name)

# Flujo principal
def flujo_principal():
    # Cargar archivo
    df = cargar_archivo()

    # Solicitar información del usuario
    usuario_info = solicitar_informacion_usuario()

    # Identificar columnas con "Pregunta"
    preguntas_cols = identificar_columnas_pregunta(df)

    # Solicitar diagnósticos primarios
    diagnosticos = solicitar_diagnosticos()

    # Calcular suma de las columnas con "Pregunta"
    df = calcular_suma(df, preguntas_cols)

    # Asignar diagnóstico primario
    df = asignar_diagnostico(df, diagnosticos)

    # Generar reporte en PDF
    generar_reporte_pdf(usuario_info, df, diagnosticos)

# Ejecutar flujo principal
flujo_principal()

Carga el archivo desde tu escritorio:


Saving Rangos de Depresion.xlsx to Rangos de Depresion (1).xlsx
Por favor, ingresa la información del docente:
Nombre del docente: D
Nombre de la asignatura: S
Aula: S
Carrera: A

Se identificaron 7 columnas con la palabra 'Pregunta':
['Pregunta 1', 'Pregunta 2', 'Pregunta 3', 'Pregunta 4', 'Pregunta 5', 'Pregunta 6', 'Pregunta 7']

¿Cuántos diagnósticos primarios hay?: 4
Ingresa el nombre del diagnóstico 1: a
Ingrese el puntaje mínimo para 'a': 0
Ingrese el puntaje máximo para 'a': 4
Ingresa el nombre del diagnóstico 2: B
Ingrese el puntaje mínimo para 'B': 5
Ingrese el puntaje máximo para 'B': 9
Ingresa el nombre del diagnóstico 3: C
Ingrese el puntaje mínimo para 'C': 10
Ingrese el puntaje máximo para 'C': 14
Ingresa el nombre del diagnóstico 4: D
Ingrese el puntaje mínimo para 'D': 15
Ingrese el puntaje máximo para 'D': 21

Diagnósticos primarios definidos:
a: 0.0 - 4.0 puntos
B: 5.0 - 9.0 puntos
C: 10.0 - 14.0 puntos
D: 15.0 - 21.0 puntos

Reporte generado: Reporte_Diagnosticos_Pr

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>